# Practico 01: Sistema de ecuaciones lineales

En este notebook resolveremos el primer práctico enfocado en métodos de resolución de sistema de ecuaciones lineales.
Los métodos que se implementarán y se compararán son:
- Método de Gauss-Jordan
- Eliminación de Gauss con backsubstitution
- Decomposición LU

Cada método será puesto a prueba con un sencillo sistema 3x3, guardando registro del tiempo de ejecución necesario para resolverlo con cada método.
Para simplificar el cómputo, cada método ha sido implementado sin pivoteo, lo cual puede generar problemas a la hora de encontrar elementos no nulos en las diagonales de las matrices.
Además, estas implementaciones serán comparadas con la función `numpy.linalg.solve`, la cual utiliza la decomposición LU con pivoteo de manera óptima.

Finalmente, utilizaremos estas implementaciones para resolver un sistema más grande.


**Importamos librerías necesarias**

In [1]:
import numpy as np

**Definimos el sistema 3x3 a resolver**

In [2]:
# Matriz de coeficientes
matrix = np.array([
    [5, -3, 2],
    [4, 6, -5],
    [8, 1, -1]
], dtype=float)

# Término independiente
independent_term = np.array([20, -2, 21], dtype=float)

Del cual conocemos su solución

In [3]:
true_solution = np.array([3, 1, 4], dtype=float)

print(matrix @ true_solution)

[20. -2. 21.]


## Gauss-Jordan

Definimos la función que implementa el método de Gauss-Jordan

In [4]:
def gauss_jordan(matrix, iterm):
    """
    Solves linear equations through Gauss-Jordan
    
    Parameters
    ----------
    matrix : 2d-array
        Matrix of coefficients
    iterm : 1d-array
        Array with independent terms
        
    Returns
    -------
    solution : 1d-array
        Solution array
    """
    # Copy matrix and independent term to avoid overwritting them
    matrix = matrix.copy()
    iterm = iterm.copy()
    # Get size of the matrix
    n = matrix.shape[0]
    for d in range(n):
        # Reduce the matrix to a diagonal matrix
        # (and also reduce the independent term)
        for row in range(n):
            if row != d:
                factor = matrix[row, d] / matrix[d, d]
                for column in range(n):
                    matrix[row, column] -= factor * matrix[d, column]
                iterm[row] -= factor * iterm[d]
    # Get solution
    return iterm / np.diag(matrix)

Comprobamos que la implementación resuelve el sistema de manera correcta

In [5]:
solution = gauss_jordan(matrix, independent_term)

print("Solucion: {}".format(solution))

assert np.allclose(solution, true_solution)

Solucion: [3. 1. 4.]


Y calculamos el tiempo de ejecución

In [6]:
%timeit gauss_jordan(matrix, independent_term)

52 µs ± 3.91 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Eliminación de Gauss con Backsubstitution

Definimos la función que implementa el método de Eliminación de Gauss con backsubstitution

In [7]:
def gauss_backsubstitution(matrix, iterm):
    """
    Solves linear equations through Gauss elimination with backsubstitution
    
    Parameters
    ----------
    matrix : 2d-array
        Matrix of coefficients
    iterm : 1d-array
        Array with independent terms
        
    Returns
    -------
    solution : 1d-array
        Solution array
    """
    # Copy matrix and independent term to avoid overwritting them
    matrix = matrix.copy()
    iterm = iterm.copy()
    # Get size of the matrix
    n = len(iterm)
    for d in range(n):
        # Reduce only the matrix elements bellow the pivot
        # (including the elements of the independent term)
        for row in range(d + 1, n):
            factor = matrix[row, d] / matrix[d, d]
            for column in range(n):
                matrix[row, column] -= factor * matrix[d, column]
            iterm[row] -= factor * iterm[d]
    # Perform backsubstitution
    solution = np.zeros_like(iterm)
    solution[-1] = iterm[-1] / matrix[-1, -1]
    for i in reversed(range(n - 1)):
        tmp = iterm[i]
        for j in range(i + 1, n):
            tmp -= matrix[i, j] * solution[j]
        tmp /= matrix[i, i]
        solution[i] = tmp
    return solution

Comprobamos que la implementación resuelve el sistema de manera correcta

In [8]:
solution = gauss_backsubstitution(matrix, independent_term)

print("Solucion: {}".format(solution))

assert np.allclose(solution, true_solution)

Solucion: [3. 1. 4.]


Y calculamos el tiempo de ejecución

In [9]:
%timeit gauss_backsubstitution(matrix, independent_term)

41.2 µs ± 2.65 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Decomposición LU

Definimos tres funciones necesarias para resolver el sistema:
1. La que aplica la decomposición LU, devuelve los elementos de las matrices L (lower) y U (upper),
2. La que aplica la forward y backsubstitution para obtener la solución del sistema a partir de un arreglo del término independiente,
3. La que obtiene la matrix LU (output de la primera función) y devuelve las dos matrices L y U.

In [10]:
def lu_decomposition(matrix):
    """
    Apply LU decomposition
    """
    n = matrix.shape[0]
    lu = matrix.copy()
    
    for j in range(n):
        for i in range(j + 1):
            tmp = 0
            for k in range(i):
                tmp += lu[i, k] * lu[k, j]
            lu[i, j] = matrix[i, j] - tmp
        for i in range(j + 1, n):
            tmp = 0
            for k in range(j):
                tmp += lu[i, k] * lu[k, j]
            lu[i, j] = (matrix[i, j] - tmp) / lu[j, j]
    return lu


def lu_solve(lu, iterm):
    """
    Solve linear equations using LU decomposed matrices
    """
    n = len(iterm)
    # Initialize the solution array as a copy of the
    # independent term.
    solution = iterm.copy()
    # Apply forward substitution
    for i in range(n):
        for k in range(i):
            solution[i] -= lu[i, k] * solution[k]
    # Apply backsubstitution
    for i in reversed(range(n)):
        for k in range(i + 1, n):
            solution[i] -= lu[i, k] * solution[k]
        solution[i] = solution[i] / lu[i, i]
    return solution


def split_lu(lu):
    """
    Split the LU matrix into L and U (lower and upper) matrices
    """
    n = lu.shape[0]
    upper = np.zeros((n, n))
    lower = np.identity(n)
    
    for i in range(n):
        for j in range(i):
            lower[i, j] = lu[i, j]
        for j in range(i, n):
            upper[i, j] = lu[i, j]
    return lower, upper

Comprobamos que la decomposición se realiza correctamente. Para lo cual verificaremos que:

$$ \mathbf{A} = \mathbf{L} \mathbf{U} $$

donde $\mathbf{A}$ es la matriz de coeficientes, y $\mathbf{L}$ y $\mathbf{U}$ son las matrices lower y upper, respectivamente.

In [11]:
lu = lu_decomposition(matrix)

L, U = split_lu(lu)

print("Matriz de coeficientes: \n{}\n".format(matrix))
print("L · U: \n{}\n".format(L @ U))

Matriz de coeficientes: 
[[ 5. -3.  2.]
 [ 4.  6. -5.]
 [ 8.  1. -1.]]

L · U: 
[[ 5. -3.  2.]
 [ 4.  6. -5.]
 [ 8.  1. -1.]]



Comprobemos ahora que las sustituciones calculan correctamente la solución del sistema

In [12]:
solution = lu_solve(lu, independent_term)

print("Solucion: {}".format(solution))

assert np.allclose(solution, true_solution)

Solucion: [3. 1. 4.]


Y calculamos el tiempo de ejecución

In [13]:
%timeit lu = lu_decomposition(matrix); lu_solve(lu, independent_term)

39.8 µs ± 2.77 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Resolución con Numpy

Comprobemos ahora que la implementación que ofrece Numpy es mucho más eficientes que las aquí presentes, además de que incorpora pivoteo para prevenir divisiones por cero.

In [14]:
%timeit np.linalg.solve(matrix, independent_term)

19.7 µs ± 1.12 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Conclusiones

Los resultados obtenidos muestran que:
- Los tres métodos son capaces de resolver correctamente el sistema de ecuaciones.
- Gauss-Jordan requiere más tiempo de ejecución que la eliminación de Gauss con backsubstitution, haciendo a esta última efectivamente más eficiente.
- La resolución del sisteam a través de la decomposición LU se realiza en aproximadamente el mismo tiempo que la eliminación de Gauss. Sin embargo, la decomposición LU nos permitiría resolver cualquier sistema que posea la misma matriz de coeficientes pero diferentes términos independientes más rápido, ya que la decomposición de la matriz debería realizarse una única vez.

## Problema NxN

Apliquemos los métodos implementados para resolver un sistema más grande.
Construiremos la matriz de coeficientes y la solución eligiendo enteros de manera aleatoria.
El término independiente lo calcularemos a partir de estas dos.

In [15]:
# Definimos tamaño del sistema, máximos valores de coeficientes y de la solución
size = 40
maxabs_coef = 20
maxabs_solution = 100

# Definimos una semilla para obtener el mismo sistema cada vez que se ejecute
np.random.seed(12345)

# Calculamos la matriz de coeficientes
matrix = np.random.randint(
    low=-maxabs_coef, high=maxabs_coef + 1, size=size**2
).reshape((size, size)).astype(float)

# Calculamos el arreglo de soluciones
true_solution = np.random.randint(
    low=-maxabs_solution, high=maxabs_solution + 1, size=size
).astype(float)

# Calculamos el término independiente como A·x
independent_term = matrix @ true_solution

print("Matriz de coeficientes: \n{}\n".format(matrix))
print("Vector de soluciones: \n{}\n".format(true_solution))
print("Término independiente: \n{}\n".format(independent_term))

Matriz de coeficientes: 
[[ 14.  17.   9. ... -12. -15.  14.]
 [-15.  -1.  -8. ... -15.  20.   0.]
 [-13. -15.  -2. ... -15. -18.  -1.]
 ...
 [ -4.  -6. -16. ...   8.   9.   1.]
 [-12.  20. -19. ... -20.  -5. -19.]
 [-17.  -3. -12. ...  13.   2.  -4.]]

Vector de soluciones: 
[ 33. -44. -92.  74.  -9. -86. -74. -80. -91. -76.  24.  -8. -63. -49.
   9.  72.  -7. -65. -27.  26. -18.  98. -81.  49.  25.  90. -30.  59.
  19.  49.   5.  67. -23.  33. -48.   8. -87.   2.  47. -68.]

Término independiente: 
[ -6737.   -258.   -165.   5683.  -3354.   1259.  -3150.   4143.   4536.
  -5148.   -604.  -3701.  -1177.   2159.  -5415.  -1766.   5275.  -2230.
   1324.   4060.   2665.  -1187.   1092.  -1668.   3047.   -571.   3814.
  -5161.  -3719.  -4066.   3175. -11290.   4036.    -29.   1398.  -3740.
   3676.  -2795.   7406.  -2330.]



Corroboremos que cada método resuelve correctamente el sistema, y registremos el tiempo de resolución

In [16]:
solution = gauss_jordan(matrix, independent_term)

print("Solucion: \n{}".format(solution))

assert np.allclose(solution, true_solution)

%timeit gauss_jordan(matrix, independent_term)

Solucion: 
[ 33. -44. -92.  74.  -9. -86. -74. -80. -91. -76.  24.  -8. -63. -49.
   9.  72.  -7. -65. -27.  26. -18.  98. -81.  49.  25.  90. -30.  59.
  19.  49.   5.  67. -23.  33. -48.   8. -87.   2.  47. -68.]
65.1 ms ± 1.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
solution = gauss_backsubstitution(matrix, independent_term)

print("Solucion: \n{}".format(solution))

assert np.allclose(solution, true_solution)

%timeit gauss_backsubstitution(matrix, independent_term)

Solucion: 
[ 33. -44. -92.  74.  -9. -86. -74. -80. -91. -76.  24.  -8. -63. -49.
   9.  72.  -7. -65. -27.  26. -18.  98. -81.  49.  25.  90. -30.  59.
  19.  49.   5.  67. -23.  33. -48.   8. -87.   2.  47. -68.]
37.8 ms ± 6.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
lu = lu_decomposition(matrix)
solution = lu_solve(lu, independent_term)

print("Solucion: \n{}".format(solution))

assert np.allclose(solution, true_solution)

%timeit lu = lu_decomposition(matrix); lu_solve(lu, independent_term)

Solucion: 
[ 33. -44. -92.  74.  -9. -86. -74. -80. -91. -76.  24.  -8. -63. -49.
   9.  72.  -7. -65. -27.  26. -18.  98. -81.  49.  25.  90. -30.  59.
  19.  49.   5.  67. -23.  33. -48.   8. -87.   2.  47. -68.]
23.9 ms ± 5.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Comparemos con la función de Numpy

In [19]:
%timeit np.linalg.solve(matrix, independent_term)

The slowest run took 9.21 times longer than the fastest. This could mean that an intermediate result is being cached.
256 µs ± 196 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
